<a href="https://colab.research.google.com/github/MohammadMahdi1128/Machine_learning/blob/main/Autoencoder_FashionMNIST_CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/MohammadMahdi1128/Machine_learning.git
%cd /content/Machine_learning

Cloning into 'Machine_learning'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 1.04 MiB | 10.90 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/Machine_learning


# Autoencoder for Fashion MNIST and CIFAR-100

## Project Overview

In this project, you will implement an Autoencoder to explore and generate hybrid images by combining feature vectors from different classes. This notebook focuses on using the **Fashion MNIST** dataset, and as a challenge, you will later apply the same methodology to the **CIFAR-100** dataset.

**Tasks:**
1. Dataset Preparation and Filtering
2. Autoencoder Implementation
3. Training the Autoencoder
4. Class Feature Centroid Calculation
5. Average Image Creation
6. Hybrid Image Generation
7. CIFAR-100 Challenge Exercise

Final Goal. Create hybrid objects. E.g. First a hybrid between a sneaker and a t-shirt and later a hybrid between a car and a plane.

**Important**: At the end you should write a report of adequate size, which will probably mean at least half a page. In the report you should describe how you approached the task. You should describe:
- Encountered difficulties (due to the method, e.g. "not enough training samples to converge", not technical like "I could not install a package over pip")
- Steps taken to alleviate difficulties
- General description of what you did, explain how you understood the task and what you did to solve it in general language, no code.
- Potential limitations of your approach, what could be issues, how could this be hard on different data or with slightly different conditions
- If you have an idea how this could be extended in an interesting way, describe it.


## Step 1: Dataset Preparation

We will work with the **Fashion MNIST** dataset, which contains 10 classes of grayscale images representing items of clothing.

### Your Tasks:
1. Load the Fashion MNIST dataset using `torchvision.datasets`.
2. Apply necessary transformations, including:
   - Normalization to scale pixel values.
   - Resizing if needed.
3. Create training and validation DataLoaders for efficient data loading.

### Hints:
- Use `torchvision.transforms` for preprocessing.
- Normalize images to have mean `0.5` and standard deviation `0.5`.

Start by writing your code to load and preprocess the dataset.

## **Please note, you can use code from https://github.com/junaidaliop/pytorch-fashionMNIST-tutorial/blob/main/pytorch_fashion_mnist_tutorial.ipynb to make it easier to load the dataset. However, whenever you copy&paste code without modifications you need to write a comment where you copied that code from.**

In [ ]:
# Import the necessary transformations module from torchvision
import torchvision.transforms as transforms

# Define a transformation pipeline.
# Here, we're only converting the images to PyTorch tensor format.
transform = transforms.Compose([transforms.ToTensor()])

# Using torchvision, load the Fashion MNIST training dataset.
# root specifies the directory where the dataset will be stored.
# train=True indicates that we want the training dataset.
# download=True will download the dataset if it's not present in the specified root directory.
# transform applies the defined transformations to the images.
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

# Create a data loader for the training set.
# It will provide batches of data, in this case, batches of size 4.
# shuffle=True ensures that the data is shuffled at the start of each epoch.
# num_workers=2 indicates that two subprocesses will be used for data loading.
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

# Similarly, load the Fashion MNIST test dataset.
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# Define the class labels for the Fashion MNIST dataset.
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

## Step 2: Autoencoder Implementation

A rudimentary implementation of an Autoencoder is given here. You may need to modify it depending on your needs. That can mean adding more convolutional layers.

Note that you need to change the kernel size and stride potentially.

Depending on your input size the Adaptive Pooling may be used on a very large feature map which can reduce the performance. You will need to figure out the sizes of the input as you add more convolutional layers. One way is to remove outputs from the encoder, for example the AdaptiveAvgPool2d and then print the output shape of the encoder. After you can stop execution for example with "assert False".

In [ ]:
# Cell 1: Import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Cell 2: Define the Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_channels=1, latent_dim=128):
        """
        Autoencoder with dynamic adjustments for varying input image dimensions.

        Parameters:
        - input_channels: Number of input channels (e.g., 1 for grayscale, 3 for RGB).
        - latent_dim: Dimensionality of the latent representation.
        """
        super(Autoencoder, self).__init__()

        # Encoder: Dynamically adapts to reduce input to a fixed latent space
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, stride=2, padding=1),  # Halve dimensions
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # Halve dimensions again
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # Further downsample
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))  # Compress to a fixed-size latent space (1x1 feature map)
        )

        # Latent space representation
        self.latent = nn.Linear(128, latent_dim)  # Flatten into 1D latent space

        # Decoder: Dynamically expands the latent representation back to the input shape
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),  # Expand back to initial channel size
            nn.Unflatten(1, (128, 1, 1)),  # Reshape to (C, H, W) for convolutional operations
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # Double dimensions
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  # Double dimensions again
            nn.ReLU(),
            nn.ConvTranspose2d(32, input_channels, kernel_size=3, stride=2, padding=1, output_padding=1),  # Final upsample
            nn.Sigmoid()  # Ensure output values are between 0 and 1
        )

    def forward(self, x):
        """
        Forward pass for the Autoencoder.

        - Encodes the input into a fixed-size latent representation.
        - Decodes the latent representation back to the input shape.
        """
        # Encode
        encoded = self.encoder(x)
        encoded = encoded.view(encoded.size(0), -1)  # Flatten to pass through linear layer
        latent = self.latent(encoded)  # Map to latent space

        # Decode
        decoded = self.decoder(latent)
        return decoded

# Cell 3: Initialize the Model
# Define hyperparameters for the model
input_channels = 1  # Example: Grayscale images
latent_dim = 128  # Latent space dimensionality
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the Autoencoder
model = Autoencoder(input_channels=input_channels, latent_dim=latent_dim).to(device)
print(model)

### Your Tasks:
1. Implement the encoder and decoder parts of the Autoencoder.
2. Ensure the model takes grayscale images (1 input channel) and outputs images of the same shape.

### Hints:
- Use `nn.Conv2d` and `nn.ConvTranspose2d`.
- Add non-linear activations like `ReLU` between layers.
- Use `Tanh` or `Sigmoid` for the final activation in the decoder.

Write your Autoencoder model below.


## Step 3: Training the Autoencoder

Train your Autoencoder to reconstruct images from the Fashion MNIST dataset.

### Your Tasks:
1. Define a suitable loss function (e.g., Mean Squared Error).
2. Set up an optimizer like Adam.
3. Write a training loop to:
   - Pass inputs through the Autoencoder.
   - Compute the reconstruction loss.
   - Backpropagate and update weights.

4. Visualize the reconstructed images periodically during training.

### Hints:
- Use GPU acceleration if available (`.cuda()`).
- Visualize outputs using `matplotlib`.

Write your training loop below.



## Step 4: Latent Space Analysis

Once the Autoencoder is trained, explore the latent space.

### Your Tasks:
1. Pass a batch of images through the encoder and store the latent representations.
2. Compute the **centroids** (average latent vectors) for each Fashion MNIST class.
3. Visualize the latent space using dimensionality reduction techniques like PCA or t-SNE.

### Hints:
- Use `sklearn.decomposition.PCA` or `sklearn.manifold.TSNE` for visualization.
- Compute centroids by averaging latent vectors of images from the same class.

Write your code to analyze the latent space below.



## Step 5: Hybrid Image Generation

Using the latent space centroids, you can create hybrid images by interpolating between centroids of two classes.

### Your Tasks:
1. Select two class centroids (e.g., "T-shirt" and "Sneaker").
2. Linearly interpolate between the centroids with a parameter `alpha` in [0, 1].
3. Decode the interpolated latent representations back into image space.

### Hints:
- Use a simple linear interpolation formula: `(1 - alpha) * centroid1 + alpha * centroid2`.
- Visualize the hybrid images for different values of `alpha`.

Write your code to generate hybrid images below.



## Step 7: Challenge Exercise: Reimplement with CIFAR-100

Now that you've successfully implemented the Autoencoder for Fashion MNIST, your next challenge is to apply the same pipeline to the **CIFAR-100 dataset**. This dataset contains 100 classes of images, each with diverse objects, making it more challenging than Fashion MNIST.

### Your Tasks:
1. Preprocess the CIFAR-100 dataset, ensuring the images are appropriately normalized and resized if needed.
2. Redefine the Autoencoder architecture to accommodate CIFAR-100's RGB images (3 channels).
3. Train the Autoencoder on CIFAR-100 and visualize the reconstructed images.
4. Compute class centroids in the latent space for selected CIFAR-100 classes (choose a manageable subset, such as 10 classes).
5. Generate hybrid images by interpolating between class centroids in the latent space.
6. Visualize the latent space clustering for the selected CIFAR-100 classes using PCA or t-SNE.

This exercise will test your understanding of the Autoencoder pipeline and challenge you to work with a more complex dataset.
